In [38]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, Flatten, Activation, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.layers import Concatenate, AveragePooling2D


# Create Inception Module

In [7]:
def inception_module(x, filter_1X1, filter_3X3_rd, filter_3X3, filter_5X5_rd, filter_5X5, filter_pool, name = None):
    conv_1X1 = Conv2D(filter_1X1, (1, 1), padding = 'same', activation = 'relu')(x)
    
    conv_3X3 = Conv2D(filter_3X3_rd, (1, 1), padding = 'same', activation = 'relu')(x)
    conv_3X3 = Conv2D(filter_3X3, (3, 3), padding = 'same', activation = 'relu')(conv_3X3)
    
    conv_5X5 = Conv2D(filter_5X5_rd, (1, 1), padding = 'same', activation = 'relu')(x)
    conv_5X5 = Conv2D(filter_5X5, (5, 5), padding = 'same', activation = 'relu')(conv_5X5)
    
    pool_proj = MaxPooling2D((3, 3), strides = (1, 1), padding = 'same')(x)
    pool_proj = Conv2D(filter_pool, (1, 1), padding = 'same', activation = 'relu')(pool_proj)
    
    output = Concatenate(axis = -1, name = name)([conv_1X1, conv_3X3, conv_5X5, pool_proj])
    
    return output

# Check in advance Inception 

In [8]:
input_tensor = Input(shape = (224, 224,  3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(input_tensor)
x = MaxPooling2D((3, 3), padding = 'same', strides = (2, 2), name = 'max_pool_1_3X3/2')(x)
x = Conv2D(64, (1, 1), padding = 'same', strides = (1, 1), activation = 'relu', name = 'conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding = 'same', strides = (1, 1), activation = 'relu', name='conv_2b_3x3/1')(x)
x = MaxPooling2D((3, 3), padding = 'same', strides = (2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x, filter_1X1=64, filter_3X3_rd= 96, filter_3X3=128, filter_5X5_rd= 16, filter_5X5= 32, filter_pool=32, name = 'inception_3a')

model = Model(inputs = input_tensor, outputs = x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 112, 112, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3X3/2 (MaxPooling2D) (None, 56, 56, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 56, 56, 64)   4160        max_pool_1_3X3/2[0][0]           
____________________________________________________________________________________________

# Making GoogLeNet(Inception Model)

In [56]:
def create_google_lenet(input_shape = (224, 224, 3), n_classes = 10):
    input_tensor = Input(input_shape)
    
    x = Conv2D(64, (7, 7), padding = 'valid', strides = (2, 2), activation = 'relu', name='conv_1_7x7/2')(input_tensor)
    x = MaxPooling2D((3, 3), strides = (2, 2), name = 'max_pool_1_3x3/2')(x)
    x = Conv2D(64, (1, 1), padding = 'same', strides = (1,1), activation = 'relu', name = 'conv_2a_3x3/1')(x)
    x = Conv2D(192, (3, 3), padding = 'same', strides = (1,1), activation = 'relu', name = 'conv_2b_3x3/1')(x)
    x = MaxPooling2D((3, 3),  strides=(2, 2), name='max_pool_2_3x3/2')(x)
    
    # Inception_3a
    x = inception_module(x, 
                         filter_1X1=64, 
                         filter_3X3_rd=96, 
                         filter_3X3=128, 
                         filter_5X5_rd=16, 
                         filter_5X5=32, 
                         filter_pool=32, 
                         name = 'Inception_3a')
    # Inception_3b
    x = inception_module(x,
                         filter_1X1=128,
                         filter_3X3_rd = 128,
                         filter_3X3=192,
                         filter_5X5_rd=32,
                         filter_5X5=96,
                         filter_pool=64,
                         name = 'Inception_3b')
    
    x = MaxPooling2D((3, 3), strides = (2, 2), name = 'max_pool_3_3x3/2')(x)
    
    # Inception_4a
    x = inception_module(x,
                         filter_1X1=192,
                         filter_3X3_rd=96,
                         filter_3X3=208,
                         filter_5X5_rd=16,
                         filter_5X5=48,
                         filter_pool=64,
                         name = 'Inception_4a')
    
    # Extra NetWork1
    x1 = AveragePooling2D((5, 5), strides = (3, 3))(x)
    x1 = Conv2D(filters = 128, kernel_size = (1, 1), padding = 'same', activation = 'relu')(x1)
    x1 = Flatten()(x1)
    x1 = Dense(1024, activation = 'relu')(x1)
    x1 = Dropout(0.7)(x1)
    x1 = Dense(n_classes, activation = 'softmax')(x1)
    
    # Inception_4b
    x = inception_module(x,
                         filter_1X1=160,
                         filter_3X3_rd=112,
                         filter_3X3=224,
                         filter_5X5_rd=24,
                         filter_5X5=64,
                         filter_pool=64,
                         name = 'Inception_4b')

    # Inception_4c
    x = inception_module(x,
                         filter_1X1=128,
                         filter_3X3_rd=128,
                         filter_3X3=256,
                         filter_5X5_rd=24,
                         filter_5X5=64,
                         filter_pool=64,
                         name = 'Inception_4c')

    # Inception_4d
    x = inception_module(x,
                         filter_1X1=112,
                         filter_3X3_rd=144,
                         filter_3X3=288,
                         filter_5X5_rd=32,
                         filter_5X5=128,
                         filter_pool=128,
                         name = 'Inception_4d')
    # Extra network 2
    x2 = AveragePooling2D(pool_size = (5,5), strides = 3)(x)
    x2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', 
                  activation = 'relu')(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1024, activation = 'relu')(x2)
    x2 = Dropout(0.7)(x2)
    x2 = Dense(n_classes, activation = 'softmax')(x2)
    
    # Inception_4e
    x = inception_module(x,
                         filter_1X1=256,
                         filter_3X3_rd=160,
                         filter_3X3=320,
                         filter_5X5_rd=32,
                         filter_5X5=128,
                         filter_pool=128,
                         name = 'Inception_4e')
    
    x = MaxPooling2D((3, 3), padding = 'same', strides = (2, 2), name = 'max_pool_4_3x3/2')(x)
    
    # inception_5a
    x = inception_module(x,
                         filter_1X1=256,
                         filter_3X3_rd=160,
                         filter_3X3=320,
                         filter_5X5_rd=32,
                         filter_5X5=128,
                         filter_pool=128,
                         name = 'inception_5a')
    # inception_5b
    x = inception_module(x,
                         filter_1X1=384,
                         filter_3X3_rd=192,
                         filter_3X3=384,
                         filter_5X5_rd=48,
                         filter_5X5=128,
                         filter_pool=128,
                         name = 'inception_5b')
    
    x = GlobalAveragePooling2D( name = 'max_pool_5_7x7/1')(x)
    x = Dropout(0.4)(x)
    output = Dense(n_classes, activation = 'softmax', name = 'output')(x)
    
    model = Model(inputs = input_tensor, outputs = [output, x1, x2], name = 'GoogLenet')
    model.summary()
    
    return model

In [57]:
model = create_google_lenet(input_shape = (224, 224, 3), n_classes = 10)

Model: "GoogLenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 109, 109, 64) 9472        input_22[0][0]                   
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 54, 54, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 54, 54, 64)   4160        max_pool_1_3x3/2[0][0]           
__________________________________________________________________________________________

In [58]:
Image_size = 128
batch_size = 64

# Learning for Model

In [79]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import Sequence
import cv2
import sklearn

def zero_one_scaler(image):
    return image / 255.0

def get_preprocessed_ohe(images, label, pre_func = None):
    if pre_func is not None:
        images = pre_func(images)
    oh_labels = to_categorical(label)
    return images, oh_labels

def get_train_valid_test(train_images, train_labels, test_images, test_labels, valid_size = 0.15, random_state = 256):
    train_images, train_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_labels = get_preprocessed_ohe(test_images, test_labels)
    
    train_images, valid_images, train_labels, valid_labels = train_test_split(train_images, train_labels, test_size = valid_size, random_state =random_state)
    
    return (train_images, train_labels), (valid_images, valid_labels), (test_images, test_labels)


class CIFAR_Dataset(Sequence):
    def __init__(self, images_array, labels, batch_size = batch_size, augmentator = None,  pre_func = None):
        self.images_array = images_array
        self.labels = labels
        self.batch_size = batch_size
        self.augmentator = augmentator
        self.pre_func = pre_func
        
    def __len__(self):
        return int(np.ceil(len(self.labels) / self.batch_size))
    
    def __getitem__(self, index):
        images_fetch = self.images_array[index*self.batch_size:(index+1)*self.batch_size]
        
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
        
        image_batch = np.zeros((images_fetch.shape[0], Image_size, Image_size, 3), dtype = 'float32')
        
        for image_index in range(images_fetch.shape[0]):
            image = cv2.resize(images_fetch[image_index], (Image_size, Image_size))
            if self.augmentator is not None:
                image = self.augmentator(image = image)['image']
            
            if self.pre_func is not None:
                image = self.pre_func(image)
            
            image_batch[image_index] = image
        
        return image_batch, label_batch

In [80]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(train_images, train_labels), (valid_images, valid_labels), (test_images, test_oh_labels) = \
    get_train_valid_test(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021)
print(train_images.shape, train_labels.shape, valid_images.shape, valid_labels.shape, test_images.shape, test_oh_labels.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(42500, 32, 32, 3) (42500, 10) (7500, 32, 32, 3) (7500, 10) (10000, 32, 32, 3) (10000, 10)


In [81]:
from tensorflow.keras.applications.inception_v3 import preprocess_input as Inception_preprocess

train_dir = CIFAR_Dataset(train_images, train_labels, pre_func= Inception_preprocess)
valid_dir = CIFAR_Dataset(valid_images, valid_labels, pre_func= Inception_preprocess)


In [82]:
model = create_google_lenet(input_shape = (128, 128, 3), n_classes = 10)

Model: "GoogLenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 61, 61, 64)   9472        input_24[0][0]                   
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 30, 30, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 30, 30, 64)   4160        max_pool_1_3x3/2[0][0]           
__________________________________________________________________________________________

In [83]:
model.compile(optimizer=Adam(learning_rate= 0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience = 3, mode = 'min', verbose = 1)
elr_cb = EarlyStopping(monitor = 'val_loss', patience= 10, mode = 'min', verbose= 1)
model.fit(train_dir, validation_data= valid_dir, epochs= 30, callbacks= [rlr_cb, elr_cb])

Epoch 1/30
665/665 [==============================] - 41s 55ms/step - loss: 5.6373 - output_loss: 1.8294 - dense_41_loss: 1.9293 - dense_43_loss: 1.8787 - output_accuracy: 0.3040 - dense_41_accuracy: 0.2505 - dense_43_accuracy: 0.2797 - val_loss: 4.7116 - val_output_loss: 1.5221 - val_dense_41_loss: 1.6466 - val_dense_43_loss: 1.5429 - val_output_accuracy: 0.4284 - val_dense_41_accuracy: 0.3793 - val_dense_43_accuracy: 0.4204
Epoch 2/30
665/665 [==============================] - 34s 52ms/step - loss: 4.5699 - output_loss: 1.4696 - dense_41_loss: 1.5994 - dense_43_loss: 1.5009 - output_accuracy: 0.4584 - dense_41_accuracy: 0.3870 - dense_43_accuracy: 0.4434 - val_loss: 4.2432 - val_output_loss: 1.3933 - val_dense_41_loss: 1.4724 - val_dense_43_loss: 1.3775 - val_output_accuracy: 0.4952 - val_dense_41_accuracy: 0.4505 - val_dense_43_accuracy: 0.5039
Epoch 3/30
665/665 [==============================] - 34s 51ms/step - loss: 3.9918 - output_loss: 1.2776 - dense_41_loss: 1.4083 - dense_43_

665/665 [==============================] - 34s 51ms/step - loss: 0.8844 - output_loss: 0.1925 - dense_41_loss: 0.4753 - dense_43_loss: 0.2166 - output_accuracy: 0.9325 - dense_41_accuracy: 0.8356 - dense_43_accuracy: 0.9252 - val_loss: 2.0973 - val_output_loss: 0.7324 - val_dense_41_loss: 0.6663 - val_dense_43_loss: 0.6986 - val_output_accuracy: 0.7945 - val_dense_41_accuracy: 0.7725 - val_dense_43_accuracy: 0.7956

Epoch 00020: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
Epoch 21/30
665/665 [==============================] - 34s 51ms/step - loss: 0.5343 - output_loss: 0.0634 - dense_41_loss: 0.3816 - dense_43_loss: 0.0893 - output_accuracy: 0.9799 - dense_41_accuracy: 0.8697 - dense_43_accuracy: 0.9737 - val_loss: 2.0860 - val_output_loss: 0.8144 - val_dense_41_loss: 0.6004 - val_dense_43_loss: 0.6712 - val_output_accuracy: 0.8251 - val_dense_41_accuracy: 0.8021 - val_dense_43_accuracy: 0.8279
Epoch 22/30
665/665 [==============================] - 34s 52ms/step

In [87]:
test_dir = CIFAR_Dataset(test_images, test_oh_labels, batch_size =  batch_size, pre_func= Inception_preprocess)
evaluate = model.evaluate(test_dir)

157/157 [==============================] - 3s 18ms/step - loss: 2.6567 - output_loss: 1.1943 - dense_41_loss: 0.6118 - dense_43_loss: 0.8506 - output_accuracy: 0.8180 - dense_41_accuracy: 0.7998 - dense_43_accuracy: 0.8206
